# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time

import langchain

# loaders
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

# prompts
from langchain import PromptTemplate, LLMChain

# vector stores
from langchain.vectorstores import Chroma, FAISS

# models
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import OpenAI

# retrievers
from langchain.chains import RetrievalQA
from langchain import PromptTemplate, LLMChain

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline



'NoneType' object has no attribute 'cadam32bit_grad_fp32'


# CFG

In [ ]:

llm = OpenAI(openai_api_key="replace with your own key",
             openai_organization="org-NP4wzeasQQUhkmhgBrG10unP")

In [3]:
class CFG:
    # # LLMs
    # model_name = 'llama2-13b' # wizardlm, bloom, falcon, llama2-7b, llama2-13b
    # temperature = 0,
    # top_p = 0.95,
    # repetition_penalty = 1.15
    
      

    # splitting
    split_chunk_size = 800
    split_overlap = 0
    
    # embeddings
    embeddings_model_repo = 'hkunlp/instructor-base'    

    # similar passages
    k = 3
    
    # paths
    PDFs_path = '/Users/MYW/Desktop/Langchain/HP books/' # replace with your own path
    Embeddings_path =  '/Users/MYW/Desktop/Langchain/public faiss index hp'
    Persist_directory = './harry-potter-vectordb'  


#  Langchain


## Files embeddings 

### Directory loader for multiple files

In [4]:
%%time

loader = DirectoryLoader(
    CFG.PDFs_path,
    glob="./*.pdf",
    loader_cls=PyPDFLoader,
    show_progress=True,
    #use_multithreading=True
)

documents = loader.load()

100%|██████████| 7/7 [01:17<00:00, 11.08s/it]

CPU times: user 1min 11s, sys: 1.1 s, total: 1min 12s
Wall time: 1min 17s


len(documents)


### Splitter
- Splitting the text into chunks so its passages are easily searchable for similarity
- This step is also only necessary if you are creating the embeddings

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = CFG.split_chunk_size,
    chunk_overlap = CFG.split_overlap
)

texts = text_splitter.split_documents(documents)
len(texts)

10519

### Embeddings
- Embedd and store the texts in a Vector database (FAISS or ChromaDB)

### Create vector database¶
- If you use Chroma vector store it will take ~35 min to create embeddings.
- If you use FAISS vector store on GPU it will take just ~3 min.
- We need to create the embeddings only once, and then we can just load the vector store and query the database using similarity search.
- Loading the embeddings takes only a few seconds.

In [ ]:
# %%time

# ### download embeddings model
# instructor_embeddings = HuggingFaceInstructEmbeddings(
#     model_name = CFG.embeddings_model_repo,
#     model_kwargs = {"device": "cuda"} # change to cpu on premise
# )

# ### create embeddings and DB
# vectordb = FAISS.from_documents(
#     documents = texts, 
#     embedding = instructor_embeddings
# )

# ### persist vector database
# vectordb.save_local("faiss_index_hp")


### Load vector database

In [6]:
%%time

### download embeddings model
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name = CFG.embeddings_model_repo,
    model_kwargs = {"device": "cpu"}
)

### load vector DB embeddings
vectordb = FAISS.load_local(
    CFG.Embeddings_path,
    instructor_embeddings
)

load INSTRUCTOR_Transformer
max_seq_length  512
CPU times: user 1.61 s, sys: 672 ms, total: 2.28 s
Wall time: 2.33 s


In [7]:
### test if vector DB was loaded correctly
vectordb.similarity_search('magic creatures')

[Document(page_content='Magical Creatures much in this weather, though as Ron said, the skrewts would probably warm them  up nicely, either by chasing \nthem, or blasting off so forcefull y that Hagrid’s cabin would catch \nfire. \nWhen they arrived at Hagrid’s cabi n, however, they found an el-\nderly witch with closely cropped gray hair and a very prominent \nchin standing before his front door. \n“Hurry up, now, the bell rang fi ve minutes ago,” she barked at \nthem as they stru ggled toward her through the snow. \n“Who’re you?” said Ron, starin g at her. “Where’s Hagrid?” \n“My name is Professor Grubbly-Pl ank,” she said briskly. “I am \nyour temporary Care of Ma gical Creatures teacher.” \n“Where’s Hagrid?” Ha rry repeated loudly. \n“He is indisposed,” said Prof essor Grubbly-Plank shortly.', metadata={'source': '/kaggle/input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 4 - The Goblet of Fire.pdf', 'page': 450}),
 Document(page_content='Regulation and Control of Mag

## Prompt Template

In [8]:
prompt_template = """
Don't try to make up an answer, if you don't know just say that you don't know.
Answer in the same language the question was asked.
Use only the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""


PROMPT = PromptTemplate(
    template = prompt_template, 
    input_variables = ["context", "question"]
)

## Retriever chain

In [10]:
retriever = vectordb.as_retriever(search_kwargs = {"k": CFG.k, "search_type" : "similarity"})

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
    retriever = retriever, 
    chain_type_kwargs = {"prompt": PROMPT},
    return_source_documents = True,
    verbose = False
)

In [11]:
# testing MMR search
question = "Which are Hagrid's favorite animals?"
vectordb.max_marginal_relevance_search(question, k = CFG.k)

[Document(page_content='“Well,\tso\tthey\tsay,”\tsaid\tHagrid.\t“Crikey,\tI’d\tlike\ta\tdragon.”', metadata={'source': '/kaggle/input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 1 - The Sorcerers Stone.pdf', 'page': 49}),
 Document(page_content="wisely. Behind him, Buckbeak spat a few ferret bones onto Hagrid'spillow.", metadata={'source': '/kaggle/input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 3 - The Prisoner of Azkaban.pdf', 'page': 228}),
 Document(page_content='HAGRID’S  TALE \n\x91 439 \x91 couple o’ creatures saved fer yer O.W.L. year, you wait, they’re some-\nthin’ really special.” \n“Erm . . . special in what way? ” asked Hermione tentatively. \n“I’m not sayin’,” said Hagrid ha ppily. “I don’ want ter spoil the \nsurprise.” \n“Look, Hagrid,” said Hermione ur gently, dropping all pretense, \n“Professor Umbridge won’t be at a ll happy if you bring anything to \nclass that’s too dangerous —” \n“Dangerous?” said Hagrid, lookin g genially bemused. “

In [12]:
# testing similarity search
question = "Which are Hagrid's favorite animals?"
vectordb.similarity_search(question, k = CFG.k)

[Document(page_content='“Well,\tso\tthey\tsay,”\tsaid\tHagrid.\t“Crikey,\tI’d\tlike\ta\tdragon.”', metadata={'source': '/kaggle/input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 1 - The Sorcerers Stone.pdf', 'page': 49}),
 Document(page_content='CHAPTER  THIRTEEN \n\x91 198 \x91 nothing better than a pet drag on, as Harry, Ron, and Hermione \nknew only too well — he had owned one for a brief period during \ntheir first year, a vicious Norweg ian Ridgeback by the name of \nNorbert. Hagrid simply loved monstrous creatures, the more \nlethal, the better. \n“Well, at least the skrewts are sma ll,” said Ron as they made their \nway back up to the castle for lunch an hour later. \n“They are now, ” said Hermione in an exasperated voice, “but \nonce Hagrid’s found out what they eat, I expect they’ll be six feet \nlong.” \n“Well, that won’t matter if they turn out to cure seasickness or \nsomething, will it?” said Ro n, grinning slyly at her. \n“You know perfectly we ll I only sa

# Post-process outputs

In [13]:
def wrap_text_preserve_newlines(text, width=200): # 110
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_llm_response(llm_response):
    ans = wrap_text_preserve_newlines(llm_response['result'])
    
    sources_used = ' \n'.join(
        [
            source.metadata['source'].split('/')[-1][:-4] + ' - page: ' + str(source.metadata['page'])
            for source in llm_response['source_documents']
        ]
    )
    
    ans = ans + '\n\nSources: \n' + sources_used
    return ans

In [14]:
def llm_ans(query):
    start = time.time()
    llm_response = qa_chain(query)
    ans = process_llm_response(llm_response)
    end = time.time()

    time_elapsed = int(round(end - start, 0))
    time_elapsed_str = f'\n\nTime elapsed: {time_elapsed} s'
    return ans + time_elapsed_str

## Ask questions

In [15]:
query = "Give me 5 examples of cool potions and explain what they do"
print(llm_ans(query))


 1. Draught of Peace: a potion to calm anxiety and soothe agitation, 2. Fizzing Whizbees: levitating sherbert balls, 3. Droobles Best Blowing Gum: fills a room with bluebell-colored bubbles that
refuse to pop for days, 4. Pepper Imps: breathe fire for your friends, 5. Ice Mice: hear your teeth chatter and squeak.

Sources: 
Harry Potter - Book 6 - The Half-Blood Prince - page: 421 
Harry Potter - Book 5 - The Order of the Phoenix - page: 247 
Harry Potter - Book 3 - The Prisoner of Azkaban - page: 164

Time elapsed: 2 s


# Gradio Chat UI

In [16]:
import gradio as gr

def predict(message, history):
    # output = message # debug mode

    output = str(llm_ans(message))
    return output

demo = gr.ChatInterface(
    predict,
    title = f' Open-Source LLM for Harry Potter Question Answering'
)

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
